<a href="https://colab.research.google.com/github/marco-siino/GM_SOURCE_CODE/blob/main/AMD_DS/Prompting_Mistral_Large_Heterogeneus_Device_Mapping_DS_Devmap_AMD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Additional experiments for EAAI Submission. Prompt engineering for device mapping (GPU vs CPU) using Mistral Large.

In [ ]:
!pip install mistralai
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from mistralai import Mistral
from tqdm import tqdm

# Inizializza Mistral API
client = Mistral(api_key='7yOu0lH5XcZC1U2ZUI2uv0ghBDduhHp5')
model = "mistral-large-latest"

# Carica dataset
df = pd.read_csv("/content/dataset-devmap-amd.csv")
df = df[["src", "oracle"]]  # src = codice, oracle = label ("CPU"/"GPU")

# Mapping etichette
text_to_label = {"CPU": 0, "GPU": 1}
label_to_text = {0: "CPU", 1: "GPU"}

# Seed per riproducibilità
random.seed(42)
np.random.seed(42)

# Setup K-Fold
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(df["src"], df["oracle"])):
    print(f"\nFold {fold + 1}")
    train_df = df.iloc[train_idx]
    val_df = df.iloc[val_idx]

    # Prompt iniziale con istruzioni
    few_shot_prompt = (
        "Classify whether the following OpenCL kernel should run on CPU or GPU based on its characteristics.\n"
        "The label should be either 'CPU' or 'GPU'.\n\n"
    )

    # Seleziona 5 esempi few-shot
    few_shot_samples = train_df.sample(n=5, random_state=fold)
    for _, row in few_shot_samples.iterrows():
        few_shot_prompt += f"Code:\n{row['src']}\nLabel: {row['oracle']}\n\n"

    preds = []
    golds = []
    texts = []

    for i, (code_snippet, true_label) in enumerate(zip(val_df["src"], val_df["oracle"])):
        print("\n######### Avvio Predizione Numero "+str(i)+" del validation set relativo al FOLD " +str(fold) + " ###########")
        golds.append(text_to_label[true_label])
        prompt = few_shot_prompt + f"Code:\n{code_snippet}\nLabel:"
        message = [{"role": "user", "content": prompt}]

        try:
            chat_response = client.chat.complete(
                model=model,
                messages=message
            )
            completion = chat_response.choices[0].message.content.strip()

            # Normalizzazione
            if "gpu" in completion.lower():
                pred_label = 1
            elif "cpu" in completion.lower():
                pred_label = 0
            else:
                pred_label = random.choice([0, 1])  # fallback

        except Exception as e:
            print(f"Errore nella predizione: {e}")
            pred_label = random.choice([0, 1])

        preds.append(pred_label)
        texts.append(completion)
        print(f"[{i}] Pred: {label_to_text[pred_label]} | True: {true_label} | GPT output: {completion}")
        print("\n######### PREDIZIONE TERMINATA. Numero "+str(i)+" del validation set relativo al FOLD " +str(fold) + " ###########")

    # Accuracy e confusion matrix
    acc = accuracy_score(golds, preds)
    print(f"Fold {fold + 1} accuracy: {acc:.4f}")
    accuracies.append(acc)

    cm = confusion_matrix(golds, preds, labels=[0, 1])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["CPU", "GPU"])
    disp.plot(cmap="Blues")
    plt.title(f"Fold {fold + 1} Confusion Matrix")
    plt.show()

# Media finale
print(f"\nAverage accuracy over all folds: {np.mean(accuracies):.4f}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 7.5 MB/s eta 0:00:00

Fold 1

######### Avvio Predizione Numero 0 del validation set relativo al FOLD 0 ###########
[0] Pred: GPU | True: GPU | GPT output: To determine whether an OpenCL kernel should run on a CPU or GPU, we need to consider several factors:

1. **Data Parallelism**: Kernels that perform the same operation on large datasets are typically better suited for GPUs.
2. **Memory Access Patterns**: GPUs are more efficient with coalesced memory accesses.
3. **Computational Intensity**: Kernels with high computational intensity (many arithmetic operations per memory access) generally perform better on GPUs.
4. **Control Flow**: Kernels with complex control flow (e.g., many branches, loops) might perform better on CPUs.
5. **Use of Local Memory**: Kernels that use local memory efficiently can benefit from GPU execution.

Let's analyze each kernel:

### Kernel 1
```c
__kernel void A(__global double* a, __global double* b,